In [1]:
import tensorflow as tf
import numpy as np
import collections
import transform
import utils
import style_transfer_tester
import vgg19

from operator import mul
from functools import reduce

class StyleTransferTrainer:
    def __init__(self, style_layer_ids, content_layer_ids, content_images, style_image, session, net, num_epochs,
                 batch_size, content_weight, style_weight, tv_weight, learn_rate, save_path, check_period, test_image):

        self.net = net
        self.sess = session

        # sort layers info
        self.CONTENT_LAYERS = collections.OrderedDict(sorted(content_layer_ids.items()))
        self.STYLE_LAYERS = collections.OrderedDict(sorted(style_layer_ids.items()))

        # input images
        self.content_list = content_images
        mod = len(content_images) % batch_size
        self.content_list = self.content_list[:]
        print(len(self.content_list))
        self.style_image = style_image
        self.content_size = len(self.content_list)

        # parameters for optimization
        self.num_epochs = num_epochs
        self.content_weight = content_weight
        self.style_weight = style_weight
        self.tv_weight = tv_weight
        self.learn_rate = learn_rate
        self.batch_size = batch_size
        self.check_period = check_period

        # path for model to be saved
        self.save_path = save_path

        # image transform network
        self.transform = transform.Transform()

        # build graph for style transfer
        self._build_graph()

        # test during training
        if test_image is not None:
            self.tester = transform.Transform('test')
            self.TEST = True

            # load content image
            print("Loading test image...")
            self.test_image = utils.load_image(test_image)

            # build graph
            self.x_test = tf.placeholder(tf.float32, shape=self.test_image.shape, name='test_input')
            self.xi_test = tf.expand_dims(self.x_test, 0)  # add one dim for batch

            # result image from transform-net
            self.transformed_test = self.tester.net(
                self.xi_test / 255.0)  # please build graph for train first. tester.net reuses variables.

        else:
            self.TEST = False

    def _build_graph(self):

        #----------------------prepare data--------------------------#

        self.batch_shape = (self.batch_size,512,512,3)

        # graph input
        print("batch shape:", self.batch_shape)
        self.content_input = tf.placeholder(tf.float32, shape=self.batch_shape, name='content')
        print("style shape:", self.style_image.shape)
        self.style_input = tf.placeholder(tf.float32, shape=self.style_image.shape, name='style')

        # preprocess for VGG
        self.content_input_pre = self.net.preprocess(self.content_input)
        self.style_input_pre = self.net.preprocess(self.style_input)

        # get content-layer-feature for content loss
        content_layers = self.net.feed_forward(self.content_input_pre, scope='content')
        self.content_layers = {}
        for id in self.CONTENT_LAYERS:
            self.content_layers[id] = content_layers[id]

        # get style-layer-feature for style loss
        style_layers = self.net.feed_forward(self.style_input_pre, scope='style')
        self.style_layers = {}
        for id in self.STYLE_LAYERS:
            self.style_layers[id] = self._gram_matrix(style_layers[id])

        # result of image transform net
        self.x = self.content_input/255.0
        self.transformed = self.transform.net(self.x)
        
        # get layer-values for x
        self.transformed_pre = self.net.preprocess(self.transformed)
        self.layers = self.net.feed_forward(self.transformed_pre, scope='mixed')

        #----------------------compute losses--------------------------#

        # style & content losses
        content_loss = 0
        style_loss = 0
        for id in self.layers:
            if id in self.CONTENT_LAYERS:
                # content loss

                features = self.layers[id]                              # content feature of this layer
                c_features = self.content_layers[id]                    # content feature of content layer

                batch, height, width, depth = features.get_shape()      # first return value is batch size (must be one)
                batch = batch.value                                     # batch size
                N = height.value*width.value                            # product of width and height
                M = depth.value                                         # number of filters

                weight = self.CONTENT_LAYERS[id]                        # weight for this layer

                content_loss += weight * 2 * tf.nn.l2_loss(features-c_features) / (batch*N*M)

            elif id in self.STYLE_LAYERS:
                # style loss

                features = self.layers[id]

                batch, height, width, depth = features.get_shape()
                batch = batch.value
                N = height.value*width.value
                M = depth.value

                weight = self.STYLE_LAYERS[id]                          # weight for this layer

                features = self._gram_matrix(features, (batch,N,M))     # style feature of this layer
                s_features = self.style_layers[id]                      # style feature of style layer

                style_loss += weight * 2 * tf.nn.l2_loss(features - s_features) / (batch * (M ** 2))

        # total variation loss
        tv_loss = self._get_total_variation_loss(self.transformed)

        # Loss of total variation regularization
        self.content_loss = self.content_weight*content_loss
        self.style_loss = self.style_weight*style_loss
        self.tv_loss = self.tv_weight*tv_loss
        self.total_loss = self.content_loss + self.style_loss + self.tv_loss

        # add summary for each loss
        tf.summary.scalar('content_loss', self.content_loss)
        tf.summary.scalar('style_loss', self.style_loss)
        tf.summary.scalar('tv_loss', self.tv_loss)
        tf.summary.scalar('total_loss', self.total_loss)

    # borrowed from https://github.com/ShafeenTejani/fast-style-transfer/blob/master/fast_style_transfer.py
    def _get_total_variation_loss(self, image):
        #Total variation loss
        shape = tuple(image.get_shape().as_list())
        tv_style_inputize = reduce(mul, (d.value for d in (image[:,1:,:,:]).get_shape()), 1)
        tv_x_size = reduce(mul, (d.value for d in (image[:,:,1:,:]).get_shape()), 1)
        tv_loss = 2 * (
                (tf.nn.l2_loss(image[:,1:,:,:] - image[:,:shape[1]-1,:,:]) /
                    tv_style_inputize) +
                (tf.nn.l2_loss(image[:,:,1:,:] - image[:,:,:shape[2]-1,:]) /
                    tv_x_size))

        return tv_loss

    def train(self):
        
        global_step = tf.train.get_or_create_global_step()

        trainable_variables = tf.trainable_variables()
        grads = tf.gradients(self.total_loss, trainable_variables)

        optimizer = tf.train.AdamOptimizer(self.learn_rate)
        train_op = optimizer.apply_gradients(zip(grads, trainable_variables), global_step=global_step,
                                             name='train_step')

        # merge all summaries into a single op
        merged_summary_op = tf.summary.merge_all()

        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(self.save_path, graph=tf.get_default_graph())

        """ session run """
        self.sess.run(tf.global_variables_initializer())

        # saver to save model
        saver = tf.train.Saver()

        # restore check-point if it exits
        checkpoint_exists = True
        try:
            ckpt_state = tf.train.get_checkpoint_state(self.save_path)
        except tf.errors.OutOfRangeError as e:
            print('Cannot restore checkpoint: %s' % e)
            checkpoint_exists = False
        if not (ckpt_state and ckpt_state.model_checkpoint_path):
            print('No model to restore at %s' % self.save_path)
            checkpoint_exists = False

        if checkpoint_exists:
            tf.logging.info('Loading checkpoint %s', ckpt_state.model_checkpoint_path)
            saver.restore(self.sess, ckpt_state.model_checkpoint_path)

        """ loop for train """
        num_examples = len(self.content_list)

        # get iteration info
        if checkpoint_exists:
            iterations = self.sess.run(global_step)
            epoch = (iterations * self.batch_size) // num_examples
            iterations = iterations - epoch*(num_examples // self.batch_size)
        else:
            epoch = 0
            iterations = 0

        while epoch < self.num_epochs:
            while iterations * self.batch_size < num_examples:
                curr = iterations * self.batch_size
                step = curr + self.batch_size
                x_batch = np.zeros(self.batch_shape, dtype=np.float32)
                for j, img_p in enumerate(self.content_list[curr:step]):
                    x_batch[j] = utils.load_image(img_p)

                iterations += 1

                assert x_batch.shape[0] == self.batch_size

                _, summary, L_total, L_content, L_style, L_tv, step = self.sess.run(
                    [train_op, merged_summary_op, self.total_loss, self.content_loss, self.style_loss, self.tv_loss, global_step],
                    feed_dict={self.content_input: x_batch, self.style_input: self.style_image})

                # write logs at every iteration
                summary_writer.add_summary(summary, iterations)
                
                print('epoch : %d, iter : %4d, ' % (epoch, step),
                      'Total Loss: %g, Content Loss: %g, Style Loss: %g, TV Loss: %g' % (L_total, L_content, L_style, L_tv))

                if step % self.check_period == 0:
                    #res = saver.save(self.sess, self.save_path + '/final.ckpt', step)

                    if self.TEST:
                        output_image = self.sess.run([self.transformed_test], feed_dict={self.x_test: self.test_image})

                        utils.save_image(output_image, self.save_path + '/result_' + "%05d" % step + '.jpg')
                        
                        
            epoch += 1
            iterations = 0
            
        res = saver.save(self.sess,self.save_path+'/final.ckpt')

    def _gram_matrix(self, tensor, shape=None):
        if shape is not None:
            B = shape[0]  # batch size
            HW = shape[1] # height x width
            C = shape[2]  # channels
            CHW = C*HW
        else:
            B, H, W, C = map(lambda i: i.value, tensor.get_shape())
            HW = H*W
            CHW = W*H*C

        # reshape the tensor so it is a (B, 2-dim) matrix
        # so that 'B'th gram matrix can be computed
        feats = tf.reshape(tensor, (B, HW, C))

        # leave dimension of batch as it is
        feats_T = tf.transpose(feats, perm=[0, 2, 1])

        # paper suggests to normalize gram matrix by its number of elements
        gram = tf.matmul(feats_T, feats) / CHW

        return gram

In [ ]:
def add_one_dim(image):
    shape = (1,) + image.shape
    return np.reshape(image, shape)

def main():
    # set device
    tf.device('/gpu:0')
    
    # initiate VGG19 model
    vgg_net = vgg19.VGG19()

    # get file list for training
    content_images = utils.get_files("content/")

    # load style image
    style_image = utils.load_image("style/Shinagawa.jpg")

    # create a map for content layers info
    content_layers = ['relu4_2']
    content_layer_weights = [1.0]
    CONTENT_LAYERS = {}
    for layer, weight in zip(content_layers,content_layer_weights):
        CONTENT_LAYERS[layer] = weight

    # create a map for style layers info
    style_layers = ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1']
    style_layer_weights = [.05, .1, .15, .3, .4]
    STYLE_LAYERS = {}
    for layer, weight in zip(style_layers, style_layer_weights):
        STYLE_LAYERS[layer] = weight

    # open session
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))

    # build the graph for train
    trainer = StyleTransferTrainer(session=sess,
                                    content_images=content_images,
                                    style_image=add_one_dim(style_image),
                                    content_layer_ids=CONTENT_LAYERS,
                                    style_layer_ids=STYLE_LAYERS,
                                    net=vgg_net,
                                    num_epochs=1000,
                                    batch_size=1,
                                    content_weight=10,
                                    style_weight=1000,
                                    tv_weight=200,
                                    learn_rate=0.0001,
                                    save_path="models/",
                                    check_period=500,
                                    test_image="content/0801.png",
                                    )
    # launch the graph in a session
    trainer.train()

    # close session
    sess.close()

if __name__ == '__main__':
    main()
    #print([tensor.name for tensor in tf.compat.v1.get_default_graph().as_graph_def().node]) 

100
batch shape: (1, 512, 512, 3)
style shape: (1, 512, 512, 3)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Loading test image...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
No model to restore at models/
epoch : 0, iter :    0,  Total Loss: 6.44032e+08, Content Loss: 6.97099e+06, Style Loss: 6.33034e+08, TV Loss: 4.0271e+06
(512, 512, 3)
epoch : 0, iter :    1,  Total Loss: 6.53951e+08, Content Loss: 6.69231e+06, Style Loss: 6.43216e+08, TV Loss: 4.04296e+06
epoch : 0, iter :    2,  Total Loss: 4.96831e+08, Content Loss: 6.21469e+06, Style Loss: 4.86614e+08, TV Loss: 4.00212e+06
epoch : 0, iter :    3,  Total Loss: 6.07

epoch : 0, iter :   58,  Total Loss: 1.75085e+08, Content Loss: 5.38198e+06, Style Loss: 1.66001e+08, TV Loss: 3.702e+06
epoch : 0, iter :   59,  Total Loss: 1.78487e+08, Content Loss: 2.08815e+06, Style Loss: 1.71539e+08, TV Loss: 4.85929e+06
epoch : 0, iter :   60,  Total Loss: 1.78465e+08, Content Loss: 7.19436e+06, Style Loss: 1.66193e+08, TV Loss: 5.07797e+06
epoch : 0, iter :   61,  Total Loss: 1.74283e+08, Content Loss: 2.82121e+06, Style Loss: 1.67074e+08, TV Loss: 4.38778e+06
epoch : 0, iter :   62,  Total Loss: 1.7749e+08, Content Loss: 5.23685e+06, Style Loss: 1.68242e+08, TV Loss: 4.01159e+06
epoch : 0, iter :   63,  Total Loss: 1.8357e+08, Content Loss: 9.24596e+06, Style Loss: 1.69774e+08, TV Loss: 4.55024e+06
epoch : 0, iter :   64,  Total Loss: 1.77684e+08, Content Loss: 3.01756e+06, Style Loss: 1.70717e+08, TV Loss: 3.94956e+06
epoch : 0, iter :   65,  Total Loss: 1.78674e+08, Content Loss: 5.41697e+06, Style Loss: 1.69086e+08, TV Loss: 4.17121e+06
epoch : 0, iter :   

epoch : 1, iter :  125,  Total Loss: 1.68826e+08, Content Loss: 4.46503e+06, Style Loss: 1.61186e+08, TV Loss: 3.17534e+06
epoch : 1, iter :  126,  Total Loss: 1.70908e+08, Content Loss: 2.90052e+06, Style Loss: 1.64837e+08, TV Loss: 3.17054e+06
epoch : 1, iter :  127,  Total Loss: 1.71646e+08, Content Loss: 6.50056e+06, Style Loss: 1.61549e+08, TV Loss: 3.59639e+06
epoch : 1, iter :  128,  Total Loss: 1.64301e+08, Content Loss: 5.6389e+06, Style Loss: 1.54446e+08, TV Loss: 4.21583e+06
epoch : 1, iter :  129,  Total Loss: 1.60903e+08, Content Loss: 4.07448e+06, Style Loss: 1.52893e+08, TV Loss: 3.93517e+06
epoch : 1, iter :  130,  Total Loss: 1.63802e+08, Content Loss: 4.66128e+06, Style Loss: 1.54492e+08, TV Loss: 4.64891e+06
epoch : 1, iter :  131,  Total Loss: 1.66756e+08, Content Loss: 6.30878e+06, Style Loss: 1.56022e+08, TV Loss: 4.42535e+06
epoch : 1, iter :  132,  Total Loss: 1.70682e+08, Content Loss: 6.81455e+06, Style Loss: 1.59176e+08, TV Loss: 4.69134e+06
epoch : 1, iter :

epoch : 1, iter :  192,  Total Loss: 1.59072e+08, Content Loss: 1.00924e+07, Style Loss: 1.44804e+08, TV Loss: 4.1753e+06
epoch : 1, iter :  193,  Total Loss: 1.82335e+08, Content Loss: 8.32354e+06, Style Loss: 1.69932e+08, TV Loss: 4.07968e+06
epoch : 1, iter :  194,  Total Loss: 1.48495e+08, Content Loss: 3.15069e+06, Style Loss: 1.41935e+08, TV Loss: 3.40927e+06
epoch : 1, iter :  195,  Total Loss: 1.47308e+08, Content Loss: 5.22201e+06, Style Loss: 1.37975e+08, TV Loss: 4.11084e+06
epoch : 1, iter :  196,  Total Loss: 1.49211e+08, Content Loss: 4.99627e+06, Style Loss: 1.40254e+08, TV Loss: 3.9598e+06
epoch : 1, iter :  197,  Total Loss: 1.48672e+08, Content Loss: 7.40263e+06, Style Loss: 1.36904e+08, TV Loss: 4.36546e+06
epoch : 1, iter :  198,  Total Loss: 1.44417e+08, Content Loss: 3.69756e+06, Style Loss: 1.37208e+08, TV Loss: 3.51176e+06
epoch : 1, iter :  199,  Total Loss: 1.58814e+08, Content Loss: 8.0431e+06, Style Loss: 1.47165e+08, TV Loss: 3.6062e+06
epoch : 2, iter :  2

epoch : 2, iter :  259,  Total Loss: 1.14256e+08, Content Loss: 3.1887e+06, Style Loss: 1.07392e+08, TV Loss: 3.67507e+06
epoch : 2, iter :  260,  Total Loss: 1.09122e+08, Content Loss: 7.76157e+06, Style Loss: 9.75176e+07, TV Loss: 3.84253e+06
epoch : 2, iter :  261,  Total Loss: 1.12463e+08, Content Loss: 3.76721e+06, Style Loss: 1.05402e+08, TV Loss: 3.29386e+06
epoch : 2, iter :  262,  Total Loss: 1.10111e+08, Content Loss: 6.06263e+06, Style Loss: 1.00756e+08, TV Loss: 3.2924e+06
epoch : 2, iter :  263,  Total Loss: 1.15034e+08, Content Loss: 9.75966e+06, Style Loss: 1.01757e+08, TV Loss: 3.51795e+06
epoch : 2, iter :  264,  Total Loss: 1.20858e+08, Content Loss: 3.52419e+06, Style Loss: 1.14584e+08, TV Loss: 2.75032e+06
epoch : 2, iter :  265,  Total Loss: 1.03203e+08, Content Loss: 5.95486e+06, Style Loss: 9.42137e+07, TV Loss: 3.034e+06
epoch : 2, iter :  266,  Total Loss: 1.04022e+08, Content Loss: 4.13520e+06, Style Loss: 9.67104e+07, TV Loss: 3.17678e+06
epoch : 2, iter :  2

...

epoch : 200, iter : 20020,  Total Loss: 7.86663e+06, Content Loss: 5.77424e+06, Style Loss: 1.76478e+06, TV Loss: 327610
epoch : 200, iter : 20021,  Total Loss: 7.31068e+06, Content Loss: 5.04442e+06, Style Loss: 1.93601e+06, TV Loss: 330255
epoch : 200, iter : 20022,  Total Loss: 8.31897e+06, Content Loss: 5.91451e+06, Style Loss: 2.06828e+06, TV Loss: 336174
epoch : 200, iter : 20023,  Total Loss: 7.50551e+06, Content Loss: 5.22367e+06, Style Loss: 1.95358e+06, TV Loss: 328255
epoch : 200, iter : 20024,  Total Loss: 1.142e+07, Content Loss: 9.34436e+06, Style Loss: 1.7371e+06, TV Loss: 338570
epoch : 200, iter : 20025,  Total Loss: 7.99863e+06, Content Loss: 5.40295e+06, Style Loss: 2.29303e+06, TV Loss: 302653
epoch : 200, iter : 20026,  Total Loss: 7.14601e+06, Content Loss: 4.69096e+06, Style Loss: 2.14021e+06, TV Loss: 314837
epoch : 200, iter : 20027,  Total Loss: 9.27139e+06, Content Loss: 6.92408e+06, Style Loss: 2.0273e+06, TV Loss: 320013
epoch : 200, iter : 20028,  Total Lo

epoch : 200, iter : 20088,  Total Loss: 7.21933e+06, Content Loss: 5.15135e+06, Style Loss: 1.74335e+06, TV Loss: 324629
epoch : 200, iter : 20089,  Total Loss: 7.14542e+06, Content Loss: 4.92781e+06, Style Loss: 1.88984e+06, TV Loss: 327764
epoch : 200, iter : 20090,  Total Loss: 9.91627e+06, Content Loss: 7.7156e+06, Style Loss: 1.86817e+06, TV Loss: 332510
epoch : 200, iter : 20091,  Total Loss: 7.41378e+06, Content Loss: 5.28422e+06, Style Loss: 1.80338e+06, TV Loss: 326187
epoch : 200, iter : 20092,  Total Loss: 1.16658e+07, Content Loss: 9.30296e+06, Style Loss: 2.01395e+06, TV Loss: 348873
epoch : 200, iter : 20093,  Total Loss: 1.35763e+07, Content Loss: 1.07339e+07, Style Loss: 2.48269e+06, TV Loss: 359789
epoch : 200, iter : 20094,  Total Loss: 6.78564e+06, Content Loss: 4.47201e+06, Style Loss: 1.99182e+06, TV Loss: 321812
epoch : 200, iter : 20095,  Total Loss: 7.60869e+06, Content Loss: 5.29909e+06, Style Loss: 1.96252e+06, TV Loss: 347082
epoch : 200, iter : 20096,  Total

epoch : 201, iter : 20156,  Total Loss: 8.38446e+06, Content Loss: 5.73932e+06, Style Loss: 2.33738e+06, TV Loss: 307754
epoch : 201, iter : 20157,  Total Loss: 7.81047e+06, Content Loss: 5.15445e+06, Style Loss: 2.33451e+06, TV Loss: 321503
epoch : 201, iter : 20158,  Total Loss: 8.57177e+06, Content Loss: 6.38294e+06, Style Loss: 1.86919e+06, TV Loss: 319639
epoch : 201, iter : 20159,  Total Loss: 6.71395e+06, Content Loss: 4.26718e+06, Style Loss: 2.15011e+06, TV Loss: 296654
epoch : 201, iter : 20160,  Total Loss: 9.49823e+06, Content Loss: 7.13166e+06, Style Loss: 2.0306e+06, TV Loss: 335965
epoch : 201, iter : 20161,  Total Loss: 7.14684e+06, Content Loss: 4.90599e+06, Style Loss: 1.90787e+06, TV Loss: 332983
epoch : 201, iter : 20162,  Total Loss: 9.03852e+06, Content Loss: 6.62339e+06, Style Loss: 2.0945e+06, TV Loss: 320632
epoch : 201, iter : 20163,  Total Loss: 1.09467e+07, Content Loss: 8.513e+06, Style Loss: 2.09793e+06, TV Loss: 335816
epoch : 201, iter : 20164,  Total Lo

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



epoch : 223, iter : 22369,  Total Loss: 7.28318e+06, Content Loss: 5.01693e+06, Style Loss: 1.96096e+06, TV Loss: 305285
epoch : 223, iter : 22370,  Total Loss: 1.15658e+07, Content Loss: 9.33711e+06, Style Loss: 1.90508e+06, TV Loss: 323613
epoch : 223, iter : 22371,  Total Loss: 8.61414e+06, Content Loss: 6.46605e+06, Style Loss: 1.83427e+06, TV Loss: 313812
epoch : 223, iter : 22372,  Total Loss: 8.68429e+06, Content Loss: 6.50726e+06, Style Loss: 1.86214e+06, TV Loss: 314890
epoch : 223, iter : 22373,  Total Loss: 6.90237e+06, Content Loss: 4.81191e+06, Style Loss: 1.78749e+06, TV Loss: 302968
epoch : 223, iter : 22374,  Total Loss: 8.62288e+06, Content Loss: 6.2063e+06, Style Loss: 2.10155e+06, TV Loss: 315037
epoch : 223, iter : 22375,  Total Loss: 8.21275e+06, Content Loss: 4.93242e+06, Style Loss: 2.96351e+06, TV Loss: 316815
epoch : 223, iter : 22376,  Total Loss: 8.57415e+06, Content Loss: 6.16684e+06, Style Loss: 2.10052e+06, TV Loss: 306788
epoch : 223, iter : 22377,  Total